<a href="https://colab.research.google.com/github/betelgeus/study/blob/master/Transformer_Using_PyTorch_(kaggle%2C_without_train%2C_eval%2C_metrics).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center" style="color:green;font-size: 3em;" >Реализация трансформера с нуля с помощью PyTorch</h1>




* [1. Введение](#section1)
* [2. Импорт библиотек](#section2)
* [3. Основные компоненты](#section3)
  - [Создание Word Embeddings](#section4)
  - [Positional Encoding](#section5)
  - [Self Attention](#section6)
* [4. Encoder](#section7)
* [5. Decoder](#section8)
* [6. Тестирование кода](#section9)
* [7. Полезные материалы](#section10)


<img src="https://theaisummer.com/static/6122618d7e1466853e88473ba375cdc7/40ffe/transformer.png">


<a class="anchor" id="section1"></a>
<h2 style="color:green;font-size: 2em;">1. Введение</h2>

В этом туториале мы поговорим о реализации трансформера из  статьи "Attention is all you need" с нуля, используя PyTorch. В основе архитектуры трансформера лежат энкодер и декодер, которые обычно используется для задач машинного перевода.


```
Примечание: Здесь мы не собираемся углубляться в подробное объяснение трансформеров. Для этого пожалуйста обратитесь к [блогу](http://jalammar.github.io/illustrated-transformer/.) Джея Аламмара. Он дал подробное объяснение внутренней работы трансформеров. Мы сосредоточимся только на технической имплементации.
```


<img src = "https://jalammar.github.io/images/t/The_transformer_encoders_decoders.png" width=600 height=400>

На изображении выше показана модель машинного перевода с французского на английский. Фактически мы можем использовать стек энкодеров (один сверху другого) и стек декодеров, как показано ниже:


<img src = "https://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" width=600 height=400>

Прежде чем продолжить, давайте взглянем на детальную картину нашей модели.

<img src = "https://miro.medium.com/max/760/1*2vyKzFlzIHfSmOU_lnQE4A.png" width=350 height=200>

<a class="anchor" id="section2"></a>
<h2 style="color:green;font-size: 2em;">2. Импорт библиотек</h2>

In [ ]:
# импорт библиотек
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import torchtext
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)

1.9.1+cpu


Мы знаем, что для задачи машинного перевода используется архитектура трансформера, включающая энкодер и декодер. Прежде чем переходить к энкодеру или декодеру, давайте обсудим некоторые основные компоненты.

<a class="anchor" id="section3"></a>
<h2 style="color:green;font-size: 2em;"> Основные компоненты</h2>

<a class="anchor" id="section4"></a>
<h2 style="color:green;"> Создание Word Embeddings</h2>

Прежде всего, нам нужно преобразовать каждое слово во входной последовательности в эмбеддинг. Векторы эмбеддингов создадут более семантическое представление каждого слова.

Предположим, что каждый эмбеддинг имеет размерность 512, и предположим, что размер нашего словаря составляет 100. Тогда наша матрица эмбеддингов будет иметь размерность 100x512. Эти матрицы будут использованы в процессе обучения, и во время вывода каждое слово будет сопоставлено соответствующему 512-мерному вектору. Предположим, что у нас есть батч размером 32 и длина последовательности 10 (10 слов). Тогда выход будет иметь размерность 32x10x512.

In [ ]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        """
        Args:
            vocab_size: размер словаря
            embed_dim: размерность эмбеддингов
        """
        super(Embedding, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)


    def forward(self, x):
        """
        Args:
            x: входящий вектор
        Returns:
            out: вектор эмбеддинга
        """
        out = self.embed(x)
        return out

<a class="anchor" id="section5"></a>
<h3 style="color:green">Positional Encoding</h3>

Следующий шаг — это позиционное кодирование. Для того чтобы модель могла понимать предложение, ей необходимо знать две вещи о каждом слове:

* что означает слово?
* какова позиция слова в предложении?

В статье "Attention is all you need" автор использует следующие функции для создания позиционного кодирования. На нечетных временных шагах используется косинусная функция, а на четных - синусная.

$$PE_{(pos, 2i)} = sin(\frac{pos}{10000^{2i/d_{model}}})$$

$$PE_{(pos, 2i+1)} = cos(\frac{pos}{10000^{2i/d_{model}}})$$


```
pos -> обозначает порядок в предложении
i -> обозначает позицию в измерении вектора вложения

```

В результате позиционного кодирования мы получим матрицу, аналогичную матрице эмбеддинга. Она будет иметь размерность "Длина последовательности (предложения)" X "Размерность эмбеддинга". Для каждого токена (слова) в последовательности мы найдем вектор эмбеддинга размерности 1 x 512 и добавим к нему соответствующий позиционный вектор размерностью 1 x 512, чтобы получить выход размерности 1 x 512 для каждого слова/токена.

Например, если у нас есть батч размером 32, длина последовательности 10 и размерность эмбединга 512, то у нас будет вектор эмбеддинга размерности 32 x 10 x 512. Точно так же у нас будет вектор позиционного кодирования размерности 32 x 10 x 512. Затем мы получим их сумму.


<img src="https://miro.medium.com/max/906/1*B-VR6R5vJl3Y7jbMNf5Fpw.png" height=200 width=400>

In [ ]:
# Буфер в PyTorch ->
# Если у вас есть параметры модели, которые должны быть сохранены и восстановлены в состоянии state_dict,
# и не обучаться оптимайзером, вы должны зарегистрировать их как буферы.


class PositionalEmbedding(nn.Module):
    def __init__(self, max_seq_len, embed_model_dim):
        """
        Args:
            max_seq_len: длина входящей последовательности
            embed_model_dim: размерность эмбеддинга
        """
        super(PositionalEmbedding, self).__init__()
        self.embed_dim = embed_model_dim

        pe = torch.zeros(max_seq_len, self.embed_dim)
        for pos in range(max_seq_len):
            for i in range(0, self.embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/self.embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/self.embed_dim)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)


    def forward(self, x):
        """
        Args:
            x: входящий вектор
        Returns:
            x: выход
        """

        # увеличиваем эмбеддинги относительно их размерности
        x = x * math.sqrt(self.embed_dim)
        # добавляем константу к эмбеддингам
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)
        return x

<a class="anchor" id="section6"></a>
<h2 style="color:green"> Self Attention</h2>

Давайте поговорим о таких ключевых компонентах трансформеров, как Self Attention and Multihead attention.

***Что такое Self Attention?***

Предположим, у нас есть последовательность "Dog is crossing the street because it saw the kitchen". К чему относится "it"? Человек легко поймет, что "it" относится к собаке. Для алгоритма машинного обучения не так все очевидно.

Когда модель обрабатывает каждое слово, Self Attention позволяет ей "смотреть" на другие слова во входной последовательности для поиска подсказок. В результате создается вектор, содержащий зависимости каждого слова от других.

Давайте пошагово рассмотрим Self Attention.


**Шаг 1:** Первый шаг при вычислении Self Attention - создать три вектора из каждого входного вектора энкодера (в данном случае эмбеддинга каждого слова). Таким образом, для каждого слова мы создаем вектор запроса (Query), вектор ключа (Key) и вектор значения (Value). Каждый из векторов будет иметь размерность 1x64.

Поскольку у нас Multihead Attention, у нас будет 8 голов Self Attention.


**Как создаются ключи (keys), запросы (queries) и значения (values)?**

Для генерации ключей, запросов и значений у нас будет матрица ключей (key matrix), матрица запросов (query matrix) и матрица значений (value matrix). Эти матрицы содержат значения, которые меняются в процессе обучения.

```
Подсказка для кода:
Предположим, у нас есть batch_size=32, sequence_length=10, embedding_dimension=512. После создания эмбеддинга слов и позиционного кодирования наш выход будет размерности 32x10x512.
Мы изменяем его размер на 32x10x8x64. (Число 8 - это количество голов в multihead attention. Если не все понятно, не беспокойтесь. Все прояснится, когда взгляните на код.)

```


**Шаг 2:** Второй шаг - это расчет оценки (score). То есть мы будем умножать матрицу запросов на матрицу ключей. [Q x K.t]

```
Подсказка для кода:
Предположим, что размерности ключей, запросов и значений составляют 32x10x8x64. Прежде чем продолжить, мы транспонируем каждую из них для удобства умножения (32x8x10x64). Теперь умножаем матрицу запросов на транспонированную матрицу ключей, то есть (32x8x10x64) x (32x8x64x10) -> (32x8x10x10).

```


**Шаг 3:** Теперь делим получившуюся матрицу на квадратный корень из размерности матрицы ключей и применяем к ней функцию Softmax. Так мы получим Scores.

```
Подсказка для кода:
Мы делим вектор размером 32x8x10x10 на 8, то есть на квадратный корень из 64 (размерности матрицы ключей).

```

**Шаг 4:** Затем умножаем Scores на матрицу значений (value matrix).

```
Подсказка для кода:
После шага 3 наш выход будет иметь размерность 32x8x10x10. Теперь умножьте его на матрицу значений (value matrix) (32x8x10x64), чтобы получить выход размерностью (32x8x10x64). Здесь 8 - это количество голов Attention, а 10 - длина последовательности. Таким образом, для каждого слова у нас есть вектор размерностью 64.

```

**Шаг 5:** Теперь мы передаем полученные значения через линейный слой, что сформирует выход Multihead Attention.

```
Подсказка для кода:
Вектор (32x8x10x64) транспонируется в (32x10x8x64), а затем изменяется размер на (32x10x512). Затем он передается через линейный слой, чтобы получить вывод размером (32x10x512).

```

Теперь у нас есть общее представление о том, как работает Multihead Attention. Давайте ознакомимся с его реализацией, чтобы прояснить детали.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim=512, n_heads=8):
        """
        Args:
            embed_dim: размерность вектора эмбеддинга на выходе
            n_heads: количество голов Self Attention
        """
        super(MultiHeadAttention, self).__init__()

        self.embed_dim = embed_dim    # размерность: 512
        self.n_heads = n_heads   # голов: 8
        self.single_head_dim = int(self.embed_dim / self.n_heads)   # 512/8 = 64. каждый key, query, value будут размерности 64

        #key,query and value matrixes  # 64 x 64
        self.query_matrix = nn.Linear(self.single_head_dim , self.single_head_dim ,bias=False)
        # одиночная матрица Key для каждого из 8 ключей  # 512x512
        self.key_matrix = nn.Linear(self.single_head_dim  , self.single_head_dim, bias=False)
        self.value_matrix = nn.Linear(self.single_head_dim ,self.single_head_dim , bias=False)
        self.out = nn.Linear(self.n_heads*self.single_head_dim ,self.embed_dim)


    def forward(self, key, query, value, mask=None):  # batch_size x sequence_length x embedding_dim    # 32 x 10 x 512
        """
        Args:
           key : вектор ключей (key vector)
           query : вектор запросов (query vector)
           value : вектор значений (value vector)
           mask: маска для декодера (mask for decoder)

        Returns:
           выходной вектор после Multihead Attention
        """
        batch_size = key.size(0)
        seq_length = key.size(1)

        # размерность запроса (query) может измениться в декодере во время inference.
        # поэтому мы не можем использовать общую длину последовательности (seq_length).
        seq_length_query = query.size(1)

        # 32x10x512
        key = key.view(batch_size, seq_length, self.n_heads, self.single_head_dim)  #batch_size x sequence_length x n_heads x single_head_dim = (32x10x8x64)
        query = query.view(batch_size, seq_length_query, self.n_heads, self.single_head_dim) #(32x10x8x64)
        value = value.view(batch_size, seq_length, self.n_heads, self.single_head_dim) #(32x10x8x64)

        k = self.key_matrix(key)       # (32x10x8x64)
        q = self.query_matrix(query)
        v = self.value_matrix(value)

        q = q.transpose(1,2)  # (batch_size, n_heads, seq_len, single_head_dim)    # (32 x 8 x 10 x 64)
        k = k.transpose(1,2)  # (batch_size, n_heads, seq_len, single_head_dim)
        v = v.transpose(1,2)  # (batch_size, n_heads, seq_len, single_head_dim)

        # рассчитаем Attention
        # скорректированные ключи (adjust key) используем для матричного умножения
        k_adjusted = k.transpose(-1,-2)  # (batch_size, n_heads, single_head_dim, seq_ken)  # (32 x 8 x 64 x 10)
        product = torch.matmul(q, k_adjusted)  # (32 x 8 x 10 x 64) x (32 x 8 x 64 x 10) = # (32x8x10x10)

        # заполняем те позиции матрицы Product значением (-1e20), где позиции маски равны 0
        if mask is not None:
             product = product.masked_fill(mask == 0, float("-1e20"))

        # делим матрицу Product на квадратный корень размерности отдельной головы (Single Head)
        product = product / math.sqrt(self.single_head_dim) # / sqrt(64)

        # применяем Softmax
        scores = F.softmax(product, dim=-1)

        # умножаем на марицу Value
        scores = torch.matmul(scores, v)  # (32x8x 10x 10) x (32 x 8 x 10 x 64) = (32 x 8 x 10 x 64)

        # сконкатинируем выходы
        concat = scores.transpose(1,2).contiguous().view(batch_size, seq_length_query, self.single_head_dim*self.n_heads)  # (32x8x10x64) -> (32x10x8x64)  -> (32,10,512)

        output = self.out(concat) #(32,10,512) -> (32,10,512)
        return output

Хорошо, теперь внезапный вопрос может возникнуть у вас в голове. Зачем эта маска используется? Не беспокойтесь, мы разберем это, когда будем говорить о декодере.

<a class="anchor" id="section7"></a>
<h2 style="color:green;font-size: 2em;"> 4. Encoder</h2>

<img src="https://www.researchgate.net/profile/Ehsan-Amjadian/publication/352239001/figure/fig1/AS:1033334390013952@1623377525434/Detailed-view-of-a-transformer-encoder-block-It-first-passes-the-input-through-an.jpg" width=300 height=200>



В блоке энкодера происходят следующие преобразования —


**Шаг 1:** : Первый вход (дополненные токены, соответствующие предложению) проходит через эмбеддинг слой и слой позиционного кодирования.


```
Подсказка для кода:
предположим, у нас есть на входе матрица размером 32x10 (batch size=32 и sequence length=10). После прохождения через эмбеддинг слой, размер матрицы становится 32x10x512. Затем матрица суммируется с соответствующим вектором позиционного кодирования, что на выходе дает матрицу размером 32x10x512. Затем этот выход передается в слой Multihead Attention.

```

**Шаг 2:** Как уже обсуждалось выше, дальше данные проходят через слой Multihead Attention, что в результате дает матрицу представлений в качестве выхода.

```
Подсказка для кода:
вход в Multihead Attention будет размером 32x10x512, из которого будут сгенерированы вектора ключей, запросов и значений, как было описано выше, и на выходе мы получаем матрицу размером 32x10x512.

```

**Шаг 3:** Затем идет нормализация и residual connection (боримся с затуханием градиентов): выход из Multihead Attention добавляется к его входу, затем нормализуется.

```
Подсказка для кода:
выход из Multihead Attention, который имеет размер 32x10x512 суммируется со входом размера 32x10x512 (который является выходом слоя эмбеддингов и позиционного кодирования), затем выход нормализуется.

```

**Шаг 4:** Затем идет полносвязанный слой (feed forward) и затем слой нормализации с residual connection от входа (вход полносвязанного слоя), куда передается выход после нормализации, и, наконец, получается выход энкодера.


```
Подсказка для кода:
Нормализованный выход будет иметь размерность 32x10x512. Он проходит через 2 линейных слоя: 32x10x512 -> 32x10x2048 -> 32x10x512. Наконец, есть residual connection, которые добавляется к выходу, и выход слоя нормализуется. Таким образом, создается вектор размером 32x10x512 в качестве выхода для энкодера.

```

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, expansion_factor=4, n_heads=8):
        super(TransformerBlock, self).__init__()
        """
        Args:
           embed_dim: размерность эмбеддингов
           expansion_factor: фактор, определяющий размерность выхода линейного слоя
           n_heads: количество Attention Heads

        """
        self.attention = MultiHeadAttention(embed_dim, n_heads)

        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

        self.feed_forward = nn.Sequential(
                          nn.Linear(embed_dim, expansion_factor*embed_dim),
                          nn.ReLU(),
                          nn.Linear(expansion_factor*embed_dim, embed_dim)
        )

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)


    def forward(self,key,query,value):
        """
        Args:
           key : вектор ключей (key vector)
           query : вектор запросов (query vector)
           value : вектор значений (value vector)
           norm2_out: выход TransformerBlock

        """

        attention_out = self.attention(key,query,value)  # 32x10x512
        attention_residual_out = attention_out + value  # 32x10x512
        norm1_out = self.dropout1(self.norm1(attention_residual_out)) # 32x10x512

        feed_fwd_out = self.feed_forward(norm1_out) # 32x10x512 -> # 32x10x2048 -> 32x10x512
        feed_fwd_residual_out = feed_fwd_out + norm1_out # 32x10x512
        norm2_out = self.dropout2(self.norm2(feed_fwd_residual_out)) # 32x10x512
        return norm2_out



class TransformerEncoder(nn.Module):
    """
    Args:
        seq_len : длина входящей последовательности
        embed_dim: размерность эмбеддингов
        num_layers: количество слоев энкодера
        expansion_factor: фактор, определяющий размерность выхода линейного полносвязанного слоя
        n_heads: количество голов в Multihead Attention

    Returns:
        out: выход Encoder'а
    """
    def __init__(self, seq_len, vocab_size, embed_dim, num_layers=2, expansion_factor=4, n_heads=8):
        super(TransformerEncoder, self).__init__()

        self.embedding_layer = Embedding(vocab_size, embed_dim)
        self.positional_encoder = PositionalEmbedding(seq_len, embed_dim)

        self.layers = nn.ModuleList([TransformerBlock(embed_dim, expansion_factor, n_heads) for i in range(num_layers)])


    def forward(self, x):
        embed_out = self.embedding_layer(x)
        out = self.positional_encoder(embed_out)
        for layer in self.layers:
            out = layer(out,out,out)

        return out  # 32x10x512

<a class="anchor" id="section8"></a>
<h2 style="color:green;font-size: 2em;"> 5. Decoder</h2>

<img src="https://discuss.pytorch.org/uploads/default/optimized/3X/8/e/8e5d039948b8970e6b25395cb207febc82ba320a_2_177x500.png" height=100 width=250>

Вот мы и рассмотрели большую часть компонентов энкодера. Давайте перейдем к компонентам декодера. Мы будем использовать выход энкодера для создания векторов ключей и значений для декодера. В декодере есть два вида Multuhead Attention: одон внутри декодера и другое между энкодером и декодером. Не беспокойтесь, мы пойдем шаг за шагом.


Давайте детально рассмотрим фазу обучения.

**Шаг 1:** Первым шагом целевая последовательность (таргет) передается через эмбеддинг слой  и слой позиционного кодирования для создания вектора эмбеддинга размерностью 1x512 для каждого слова в целевой последовательности.


```
Подсказка для кода:
Предположим, что длина последовательности составляет 10, размер батча - 32, а размерность вектора эмбеддинга - 512. У нас есть вход размером 32x10 для матрицы эмбеддингов, которая дает выход размером 32x10x512, который добавляется к позиционному кодированию той же размерности, и создается выход размером 32x10x512.
```

**Шаг 2:** Выход после эмбеддинг слоя  проходит через слои Multihead Attention, как и раньше (создание матриц: ключей, запросов и значений из таргета). В результате прохождения через Multihead Attention мы получаем вектор, который содержит информацию о взаимосвязи (внимании) слов между собой. В этот раз основное отличие заключается в том, что мы используем маску.

**Зачем нужна маска?**

Маска нужна, чтобы слово не смотрело на будущие слова при проверки зависимости. Поскольку мы создаем Attention для слов в целевой последовательности, нам не нужно, чтобы определенное слово видело будущие слова. Например, в фразе "Я студент", нам не нужно, чтобы слово "Я" видело слово "студент".


```
Подсказка для кода:
Для создания Attention мы создали треугольную матрицу с 1 и 0. Например, треугольная матрица для длины последовательности 5 выглядит следующим образом:

1 0 0 0 0
1 1 0 0 0
1 1 1 0 0
1 1 1 1 0
1 1 1 1 1

После умножения ключей (keys) на запросы (values) мы заполняем все позиции со значением 0 отрицательной бесконечностью. В коде мы заполняем это очень маленьким числом, чтобы избежать ошибок деления (с помощью -1e20).
```

**Шаг 3:** Как и раньше, у нас есть слои Add и Norm, где выход эмбеддинг слоя добавляется к результату Attentions и нормализуется.



**Шаг 4:** Затем у нас есть еще один слой Multihead Attention, а затем слой Add и Norm. Это Multihead Attention называется Multihead Attention между декодером и энкодером. Для этого Multihead Attention мы создаем векторы ключей (keys) и  значений (values) из выхода энкодера. Запрос (query) создается из выхода предыдущего слоя декодера.

```
Подсказка для кода:
Таким образом, у нас есть выход размером 32x10x512 из энкодера. Ключи и значения для всех слов создаются из него. Точно так же матрица запросов создается из выхода предыдущего слоя декодера (32x10x512).
```

Затем выход проходит через слой Multihead Attention (мы используем количество голов = 8), затем через слой Add и Norm. Здесь вывод из предыдущего слоя энкодера (то есть выход из предыдущего слоя Add и Norm) добавляется к выходу между декодером и энкодером и затем нормализуется.


**Шаг 5:** Далее у нас есть полносвязанный линейный слой  (feed forward) с Add и Norm, который аналогичен тому, что есть в энкодере.


**Шаг 6:** Наконец, мы создаем линейный слой длиной, равной количеству слов в общем целевой корпусе, и добавляем к нему функцию softmax, чтобы получить вероятность для каждого слова.

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, expansion_factor=4, n_heads=8):
        super(DecoderBlock, self).__init__()
        """
        Args:
           embed_dim: размерность эмбеддингов
           expansion_factor: фактор, определяющий размерность выхода линейного полносвязанного слоя
           n_heads: количество Attention Head

        """
        self.attention = MultiHeadAttention(embed_dim, n_heads=8)
        self.norm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(0.2)
        self.transformer_block = TransformerBlock(embed_dim, expansion_factor, n_heads)


    def forward(self, key, query, x, mask):
        """
        Args:
           key : вектор ключей (key vector)
           query : вектор запросов (query vector)
           value : вектор значений (value vector)
           mask: маска для декодера (mask for decoder)
           key: key vector
           query: query vector
           value: value vector
           mask: маска для Multihead Attention
        Returns:
           out: выход TransformerBlock

        """

        # мы должны использовать маску только для первого Attention
        attention = self.attention(x, x, x, mask=mask) # 32x10x512
        value = self.dropout(self.norm(attention + x))

        out = self.transformer_block(key, query, value)
        return out



class TransformerDecoder(nn.Module):
    def __init__(self, target_vocab_size, embed_dim, seq_len, num_layers=2, expansion_factor=4, n_heads=8):
        super(TransformerDecoder, self).__init__()
        """
        Args:
           target_vocab_size: размерность корпуса документов таргета
           embed_dim: размерность эмбеддингов
           seq_len : длина входящей последовательности
           num_layers: количество слоев Encoder'а
           expansion_factor: фактор, который определяет количество слоев в полносвязанных линейных слоях
           n_heads: количество голов in Multihead Attention

        """
        self.word_embedding = nn.Embedding(target_vocab_size, embed_dim)
        self.position_embedding = PositionalEmbedding(seq_len, embed_dim)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_dim, expansion_factor=4, n_heads=8)
                for _ in range(num_layers)
            ]

        )
        self.fc_out = nn.Linear(embed_dim, target_vocab_size)
        self.dropout = nn.Dropout(0.2)


    def forward(self, x, enc_out, mask):
        """
        Args:
            x: таргетный вектор, который подается на вход модели
            enc_out : выходной вектор Encoder'а
            trg_mask: маска для Decoder'a Self Attention
        Returns:
            out: вектор выхода
        """


        x = self.word_embedding(x)  # 32x10x512
        x = self.position_embedding(x) # 32x10x512
        x = self.dropout(x)

        for layer in self.layers:
            x = layer(enc_out, x, enc_out, mask)

        out = F.softmax(self.fc_out(x))
        return out

Наконец, мы упорядочим все подмодули и создадим полную архитектуру трансформера.

In [ ]:
class Transformer(nn.Module):
    def __init__(self, embed_dim, src_vocab_size, target_vocab_size, seq_length, num_layers=2, expansion_factor=4, n_heads=8):
        super(Transformer, self).__init__()
        """
        Args:
           embed_dim:  размерность эмбеддингов
           src_vocab_size: размер корпуса документов source
           target_vocab_size: размер корпуса документов target
           seq_length : длина входящей последовательности
           num_layers: количестов  слоев Encoder'а
           expansion_factor: фактор, который определяет количество слоев в полносвязанных линейных слоях
           n_heads: количество голов in Multihead Attention

        """

        self.target_vocab_size = target_vocab_size

        self.encoder = TransformerEncoder(seq_length, src_vocab_size, embed_dim, num_layers=num_layers, expansion_factor=expansion_factor, n_heads=n_heads)
        self.decoder = TransformerDecoder(target_vocab_size, embed_dim, seq_length, num_layers=num_layers, expansion_factor=expansion_factor, n_heads=n_heads)


    def make_trg_mask(self, trg):
        """
        Args:
            trg: последовательность target'а
        Returns:
            trg_mask: маска target'а
        """
        batch_size, trg_len = trg.shape
        # возвращает нижнюю треугольную часть матрицы, заполненную единицами
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            batch_size, 1, trg_len, trg_len
        )
        return trg_mask


    def decode(self,src,trg):
        """
        для инференса
        Args:
            src: вход в Encoder
            trg: вход в Decoder
        out:
            out_labels : возвращает итоговый предикт последовательности
        """
        trg_mask = self.make_trg_mask(trg)
        enc_out = self.encoder(src)
        out_labels = []
        batch_size,seq_len = src.shape[0],src.shape[1]
        # outputs = torch.zeros(seq_len, batch_size, self.target_vocab_size)
        out = trg

        for i in range(seq_len): # 10
            out = self.decoder(out,enc_out,trg_mask) #bs x seq_len x vocab_dim
            # возьмем последний токен
            out = out[:,-1,:]

            out = out.argmax(-1)
            out_labels.append(out.item())
            out = torch.unsqueeze(out,axis=0)

        return out_labels


    def forward(self, src, trg):
        """
        Args:
            src: вход в Encoder
            trg: вход в Decoder
        out:
            out: итоговый вектор с вероятностями каждого таргетного слова
        """
        trg_mask = self.make_trg_mask(trg)
        enc_out = self.encoder(src)

        outputs = self.decoder(trg, enc_out, trg_mask)
        return outputs

<a class="anchor" id="section9"></a>
<h2 style="color:green;font-size: 2em;"> 6. Тестирование кода </h2>

Предположим, у нас есть входная последовательность длиной 10 и целевая последовательность длиной 10.

In [ ]:
src_vocab_size = 11
target_vocab_size = 11
num_layers = 6
seq_length= 12


# пусть 0 будет sos токеном и 1 будет eos токеном
src = torch.tensor([[0, 2, 5, 6, 4, 3, 9, 5, 2, 9, 10, 1],
                    [0, 2, 8, 7, 3, 4, 5, 6, 7, 2, 10, 1]])
target = torch.tensor([[0, 1, 7, 4, 3, 5, 9, 2, 8, 10, 9, 1],
                       [0, 1, 5, 6, 2, 4, 7, 6, 2, 8, 10, 1]])

print(src.shape,target.shape)

torch.Size([2, 12]) torch.Size([2, 12])


In [ ]:
model = Transformer(embed_dim=512, src_vocab_size=src_vocab_size,
                    target_vocab_size=target_vocab_size, seq_length=seq_length,
                    num_layers=num_layers, expansion_factor=4, n_heads=8)

In [ ]:
model

In [ ]:
out = model(src, target)
out.shape

torch.Size([2, 12, 11])

In [ ]:
# инференс
src = torch.tensor([[0, 2, 5, 6, 4, 3, 9, 5, 2, 9, 10, 1]])
trg = torch.tensor([[0]])
print(src.shape,trg.shape)
out = model.decode(src, trg)
out

torch.Size([1, 12]) torch.Size([1, 1])


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

**Важно: В коде реализован только feed forward.**


<a class="anchor" id="section10"></a>
<h2 style="color:green;font-size: 2em;"> 10.  Полезные материалы</h2>

* Understanding transformers
  - https://theaisummer.com/transformer/
  - https://jalammar.github.io/illustrated-transformer/
* Pytorch implementation
  - https://www.youtube.com/watch?v=U0s0f995w14